In [1]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter

from dotenv import load_dotenv
from kiwipiepy import Kiwi
import os
from huggingface_hub import login

# Hugging Face 로그인 코드
def huggingface_login():
    login(token=os.getenv("HUGGING_FACE_API_KEY"))

huggingface_login()
load_dotenv()
kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)] 


endpoint = "http://0.0.0.0:6006/v1/traces"
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

/Users/lyw/anaconda3/envs/multiagent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.docstore.document import Document
from smolagents import CodeAgent, HfApiModel, ToolCallingAgent, MultiStepAgent, OpenAIServerModel
from src.tools import GetResponse, OpenAPIGuideRetriever
        
def list_pdf_files(directory: str) -> list:
    """
    주어진 디렉토리에서 PDF 파일 목록(전체 경로)을 반환합니다.
    """
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.lower().endswith('.pdf')]

def load_pdf_documents(pdf_files: list) -> list:
    """
    PDF 파일 목록을 받아 각 파일을 Document 객체로 변환합니다.
    PyMuPDFLoader를 사용하여 PDF 파일의 모든 페이지 텍스트를 추출하고,
    이를 하나의 Document 객체로 결합합니다.
    """
    documents = []
    for pdf_path in pdf_files:
        try:
            loader = PDFPlumberLoader(pdf_path)
            docs = loader.load()  # 각 페이지별 Document 객체 리스트
            # 모든 페이지 텍스트를 하나로 결합
            full_text = "\n".join([doc.page_content for doc in docs])
            document = Document(page_content=full_text, metadata={"source": pdf_path})
            documents.append(document)
        except Exception as e:
            print(f"Error loading {pdf_path}: {e}")
    return documents

openapi_dir = '..docs/openapi/'
pdf_files = list_pdf_files(openapi_dir)

print(f"{len(pdf_files)}개의 PDF 파일을 발견하였습니다.")
docs = load_pdf_documents(pdf_files)
print(f"총 {len(docs)}개의 Document 객체를 생성하였습니다.")

model = HfApiModel()
model = OpenAIServerModel(model_id='gpt-4o-mini')

get_response = GetResponse()
open_api_guide_retriever = OpenAPIGuideRetriever(docs)
tools =[open_api_guide_retriever,get_response]

tool_agent = ToolCallingAgent(
    model=model,
    tools=tools,
    name = "OpenAPI guide document retrieve agent",
    description="""
    You are an agent that reads the API guide document and finds the necessary information to report. The report must include the Call Back URL (not the service URL), request parameter, response parameter, and response message JSON structure.
    """,
    max_steps = 1
)
rag_agent = MultiStepAgent(
    tools = tools,
    model = model,
    name="managed_agent",
    description="""
    You are an agent that reads the API guide document and finds the necessary information to report. The report must include the callback URL, request parameter, response parameter, and response message JSON structure.
    """,
    managed_agents=[tool_agent],
    max_steps=3
)
LH_agent = CodeAgent(
    tools=tools,
    model=model,
    additional_authorized_imports=["json","requests"],
    use_e2b_executor=False,
    max_steps = 6
)

open_api_query = '분양임대공고조회'
LH_opt = {'지역' : "서울",
       '기간' : "2024.12.01 ~ 2025.04.10",
       '청약유형' : "주택 임대",
       '공고상태' : ""
       }

#open_api_parameter = tool_agent.run("분양임대공고조회")
#rag_output = rag_agent.run(f"{open_api_query} 관련 문서를 참고해서 API 호출에 필요한 callback url, request 파라미터, response로 받는 파라미터를 작성해서 보고하는데 포맷은 조회한 문서 이름 : 문서이름, callback url : callback url, resquest parameter : parameters")
final_answer = LH_agent.run(task=f'조건(지역, 기간, 청약 유형)을 고려한 주택 임대 공고를 찾기 위해 get_response tool을 호출하여 공고의 공고 제목, 공고문 상세 주소, 공고문 상세 내용(시작일, 마감일, 상태, 유형, 지역)을 보기좋게 정리해서 답변해줘',additional_args=LH_opt,stream=False)

5개의 PDF 파일을 발견하였습니다.
총 5개의 Document 객체를 생성하였습니다.


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 조건(지역, 기간, 청약 유형)을 고려한 주택 임대 공고를 찾기 위해 get_response tool을 호출하여 공고의 공고 제목,  │
│ 공고문 상세 주소, 공고문 상세 내용(시작일, 마감일, 상태, 유형, 지역)을 보기좋게 정리해서 답변해줘               │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'지역': '서울', '기간': '2024.12.01 ~ 2025.04.10', '청약유형': '주택 임대', '공고상태': ''}.                   │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # First, let's find the API documentation for the housing rental announcements query.                            
  api_guide = open_api_guide_retriever(query="분양임대공고조회")                                                   
  print(api_guide)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

Retrieved documents:


===== Document 0 =====
{'source': 'openapi/OpenAPI활용가이드_한국토지주택공사_분양임대공고조회_20241010.pdf'}공공데이터 오픈 API 
활용가이드

Open API 활용가이드
목 차
1. 서비스 명세 
...................................................................................................................
........................................ 3
1.1 분양임대공고문 조회 서비스 
...................................................................................................................
........................ 3
가. API 서비스 개요 
...................................................................................................................
.......................................... 3
나. 상세기능 목록 
...................................................................................................................
.............................................. 4
다. 상세기능내역 
...................................................................................................................
................................................ 4
1) [분양임대공고문 조회] 상세기능명세 4
2. OpenAPI 에러 코드정리 
...................................................................................................................
................... 9
- 2 -

Open API 활용가이드
1. 서비스 명세
1.1 분양임대공고문 조회 서비스
가. API 서비스 개요
API명(영문) lhLeaseNoticeInfo1
API명(국문) 분양임대공고문 조회 서비스
API 서비스 정보 광역시도 코드, 공고유형코드, 공고상태코드, 공고명으로
API 설명 분양.임대공고문을 이용하여 공고유형, 공고명, 지역,
공고게시일 정보조회 서비스
[O] serviceKey [ ] 인증서 (GPKI/NPKI)
서비스 인증/권한
[ ] Basic (ID/PW) [ ] 없음
메시지 레벨
[ ] 전자서명 [ ] 암호화 [O] 없음
암호화
전송 레벨 암호화 [ ] SSL [O] 없음
API 서비스
[ ] SOAP 1.2
보안적용
(RPC-Encoded, Document Literal, Document Literal
기술 수준
인터페이스 표준 Wrapped)
[O] REST (GET)
[ ] RSS 1.0 [ ] RSS 2.0 [ ] Atom 1.0 [ ] 기타
교환 데이터 표준
[] XML [O] JSON [ ] MIME [ ] MTOM
(중복선택가능)
서비스 URL http://apis.data.go.kr/B552555/lhLeaseNoticeInfo1
서비스 명세 URL http://apis.data.go.kr/B552555/lhLeaseNoticeInfo1?_wadl&ty
(WSDL 또는 pe=json
WADL)
서비스 버전 1.0
API 서비스
서비스 시작일 2019-08-01 서비스 배포일 2019-08-01
배포정보
서비스 이력 2019-08-01 : 서비스 시작
메시지 교환유형 [O] Request-Response [ ] Publish-Subscribe
[ ] Fire-and-Forgot [ ] Notification
서비스 제공자
데이터 갱신주기 수시
- 3 -

Open API 활용가이드
나. 상세기능 목록
번호 API명(국문) 상세기능명(영문) 상세기능명(국문)
1 분양임대공고문 조회 서비스 LhLeaseNoticeInfo1 분양임대공고문 조회
다. 상세기능내역
1) [분양임대공고문 조회] 상세기능명세
a) 상세기능정보
상세기능 번호 1 상세기능 유형 조회 (목록)
상세기능명(국문) 분양임대공고문 조회
상세기능 설명 광역시도 코드, 공고유형코드, 공고상태코드, 공고명으로 분양.임대공고문을
이용하여 공고유형, 공고명, 지역, 공고게시일 정보를 조회하는 목록 조회
기능
Call Back URL http://apis.data.go.kr/B552555/lhLeaseNoticeInfo1/lhLeaseNoticeInfo1
최대 메시지 사이즈 [4000] byte
평균 응답 시간 [500] ms 초당 최대 트랙잭션 [30] tps
b) 요청 메시지 명세
항목명(영문) 항목명(국문) 항목크기 항목구분 샘플데이터 항목설명
serviceKey 인증키 100 1 인증키 공공데이터포털에서
(URL Encode) 발급받은 인증키
PG_SZ 한 페이지 결과 4 1 10 한 페이지 결과 수
수
PAGE 페이지 번호 4 1 1 페이지 번호
PAN_NM 공고명 50 0 대전 공고명으로 조회
UPP_AIS_TP_CD 공고유형코드 10 0 01 공고유형코드
CNP_CD 지역코드 10 0 11 지역코드
PAN_SS 공고상태코드 10 0 공고중 공고상태코드
PAN_NT_ST_DT 공고게시일 8 1 20190723
CLSG_DT 공고마감일 8 1 20190822
※ 항목구분 : 필수(1), 옵션(0)
- 4 -

Open API 활용가이드
공고유형코드(UPP_AIS_TP_CD)
코드 설명
01 토지
05 분양주택
06 임대주택
13 주거복지
22 상가
39 신혼희망타운
지역코드(CNP_CD)
코드 설명
11 서울특별시
26 부산광역시
27 대구광역시
28 인천광역시
29 광주광역시
30 대전광역시
31 울산광역시
36110 세종특별자치시
41 경기도
42 강원도
43 충청북도
44 충청남도
52 전북특별자치도
46 전라남도
47 경상북도
48 경상남도
50 제주특별자치도
공고상태코드(PAN_SS)
코드 설명
공고중 공고중
접수중 접수중
접수마감 접수마감
상담요청 상담요청
정정공고중 정정공고중
- 5 -

Open API 활용가이드
c) 응답 메시지 명세
항목명(영문) 항목명(국문) 항목크기 항목구분 샘플데이터 항목설명
SS_CODE 결과코드 2 1 Y 결과코드
RS_DTTM 출력일시 20 1 20241010045535 출력일시
RNUM 순번 10 1 1 데이터순번
UPP_AIS_TP_N 공고유형명 10 1 임대주택 공고유형명
M
AIS_TP_CD_NM 공고세부유형명 10 1 행복주택 공고세부유형명
PAN_NM 공고명 100 1 행복도시 3- 데이터 조회를 위한
1M5 블록 10 년 키값
공공임대주택리츠
CNP_CD_NM 지역명 50 1 전국 지역명
PAN_SS 공고상태 10 1 공고중 공고상태
ALL_CNT 전체조회건수 10 1 21710 전체조회건수
DTL_URL 공고상세URL 100 1 https://apply.lh.or. 공고상세URL
kr/LH/index.html?
gv_url=SIL::CLCC_
SIL_0050.xfdl&gv_
menuId=1010203
&gv_param=CCR
_CNNT_SYS_DS_C
D:02,PAN_ID:0000
059133,LCC:Y
※ 항목구분 : 필수(1), 옵션(0)
d) 요청/응답 메시지 예제
요청메시지
http://apis.data.go.kr/B552555/lhLeaseNoticeInfo1/lhLeaseNoticeInfo1?serviceKey=인증키(URL
Encode)&PG_SZ=10&PAGE=1

...

[Step 2: Duration 6.17 seconds| Input tokens: 19,126 | Output tokens: 799]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Prepare the final structured output                                                                            
  final_output = """                                                                                               
  1. 강남에버시움(서울강남3), 서울서초3 국민임대주택 예비입주자 모집공고                                           
     - 상세 주소:                                                                                                  
  [Link](https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300017662&ccrCnntSysDsCd  
  =03&uppAisTpCd=06&aisTpCd=07&mi=1026)                                                                            
     - 시작일: 2025.03.14                                                                                          
     - 마감일: 2025.03.27                                                                                          
     - 상태: 접수마감                                                                                              
     - 유형: 국민임대                                                                                              
     - 지역: 서울특별시                                                                                            
                                                                                                                   
  2. 관악봉천 H-1ㆍ2ㆍ3BL 행복주택 입주자격완화 추가모집                                                           
     - 상세 주소:                                                                                                  
  [Link](https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300017643&ccrCnntSysDsCd  
  =03&uppAisTpCd=06&aisTpCd=10&mi=1026)                                                                            
     - 시작일: 2025.03.13                                                                                          
     - 마감일: 2025.03.26                                                                                          
     - 상태: 접수마감                                                                                              
     - 유형: 행복주택                                                                                              
     - 지역: 서울특별시                                                                                            
                                                                                                                   
  3. [정정공고]2025년 2월 세종시 국민임대주택 예비입주자 모집                                                      
     - 상세 주소:                                                                                                  
  [Link](https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300017597&ccrCnntSysDsCd  
  =03&uppAisTpCd=06&aisTpCd=07&mi=1026)                                                                            
     - 시작일: 2025.03.05                                                                                          
     - 마감일: 2025.03.13                                                                                          
     - 상태: 접수마감                                                                                              
     - 유형: 국민임대                                                                                              
     - 지역: 세종특별자치시                                                                                        
                                                                                                                   
  4. 2025년 2월 세종시 국민임대주택 예비입주자 모집                                                                
     - 상세 주소:                                                                                                  
  [Link](https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300017591&ccrCnntSysDsCd  
  =03&u

Out - Final answer: 
1. 강남에버시움(서울강남3), 서울서초3 국민임대주택 예비입주자 모집공고
   - 상세 주소: 
[Link](https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300017662&ccrCnntSysDsCd=03&
uppAisTpCd=06&aisTpCd=07&mi=1026)
   - 시작일: 2025.03.14
   - 마감일: 2025.03.27
   - 상태: 접수마감
   - 유형: 국민임대
   - 지역: 서울특별시

2. 관악봉천 H-1ㆍ2ㆍ3BL 행복주택 입주자격완화 추가모집
   - 상세 주소: 
[Link](https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300017643&ccrCnntSysDsCd=03&
uppAisTpCd=06&aisTpCd=10&mi=1026)
   - 시작일: 2025.03.13
   - 마감일: 2025.03.26
   - 상태: 접수마감
   - 유형: 행복주택
   - 지역: 서울특별시

3. [정정공고]2025년 2월 세종시 국민임대주택 예비입주자 모집
   - 상세 주소: 
[Link](https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300017597&ccrCnntSysDsCd=03&
uppAisTpCd=06&aisTpCd=07&mi=1026)
   - 시작일: 2025.03.05
   - 마감일: 2025.03.13
   - 상태: 접수마감
   - 유형: 국민임대
   - 지역: 세종특별자치시

4. 2025년 2월 세종시 국민임대주택 예비입주자 모집
   - 상세 주소: 
[Link](https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300017591&ccrCnntSysDsCd=03&
uppAisTpCd=06&aisTpCd=07&mi=1026)
   - 시작일: 2025.02.27
   - 마감일: 2025.03.13
   - 상태: 접수마감
   - 유형: 국민임대
   - 지역: 세종특별자치시

5. [정정공고]2024년 서울특별시 영구임대주택 예비입주자 모집
   - 상세 주소: 
[Link](https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300017328&ccrCnntSysDsCd=03&
uppAisTpCd=06&aisTpCd=09&mi=1026)
   - 시작일: 2024.12.13
   - 마감일: 2024.12.30
   - 상태: 접수마감
   - 유형: 영구임대
   - 지역: 서울특별시

6. 2024년 서울특별시 영구임대주택 예비입주자 모집
   - 상세 주소: 
[Link](https://apply.lh.or.kr/lhapply/apply/wt/wrtanc/selectWrtancInfo.do?panId=2015122300017310&ccrCnntSysDsCd=03&
uppAisTpCd=06&aisTpCd=09&mi=1026)
   - 시작일: 2024.12.13
   - 마감일: 2024.12.30
   - 상태: 접수마감
   - 유형: 영구임대
   - 지역: 서울특별시

[Step 3: Duration 26.78 seconds| Input tokens: 30,380 | Output tokens: 2,706]

In [63]:
from smolagents import OpenAIServerModel
from src.tools import SH_NoticeScraper
import datetime
from smolagents import CodeAgent, HfApiModel, ToolCallingAgent

model = OpenAIServerModel(model_id='gpt-4o-mini')
sh_notice_scraper = SH_NoticeScraper()
sh_url = "https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv="
sh_agent = CodeAgent(
    description=
    """
    sh_notice_scraper를 사용해서 웹사이트의 게시판 글 목록을 스크래핑하여, 조건(예: 제목에 '장기미임대' 포함, 기간이 '2024-01-01 ~ 2025-05-05')에 맞게 게시글 정보를 정리해서 반환하는 agent.
    """,
    tools=[sh_notice_scraper],
    model=model,
    use_e2b_executor=False,
    max_steps = 6
)
opt = {}
opt['조건']= {'지역' : "서울",
      '기간' : "2024.12.01 ~ 2025.04.10",
      '청약유형' : "",
      'keyword' : "",
      'sh_url' : "https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv=",
      '공고상태' : ""
      }
today = datetime.date.today()
task_prompt = f"""
<STEP 1>: max_pages(int) 수는 조건기간, 1 pages per 3 month를 고려해서 sh_notice_parser를 호출하고 그 결과를 keyword로 요청이 있지 않으면 '공고명'에 '발표','합격자','경쟁률','결과'가 있는 결과는 필터링해. STEP 1 코드 한번에 작성. </STEP 1>
<STEP 2>: !important 조건(기간, 청약유형, 공고상태)들을 고려해서 결과를 다시 필터링. '공고게시일'이 기간안에 반드시 포함되어야한다. 만약 공고상태나 청약유형 없으면 고려안해도 된다. STEP 2 코드 한번에 작성.</STEP 2>
"""
sh_final_answer = sh_agent.run(task=task_prompt, additional_args=opt, stream=False)
#print(sh_final_answer)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ <STEP 1>: max_pages(int) 수는 조건기간, 1 pages per 3 month를 고려해서 sh_notice_parser를 호출하고 그 결과를    │
│ keyword로 요청이 있지 않으면 '공고명'에 '발표','합격자','경쟁률','결과'가 있는 결과는 필터링해. STEP 1 코드     │
│ 한번에 작성. </STEP 1>                                                                                          │
│ <STEP 2>: !important 조건(기간, 청약유형, 공고상태)들을 고려해서 결과를 다시 필터링. '공고게시일'이 기간안에    │
│ 반드시 포함되어야한다. 만약 공고상태나 청약유형 없으면 고려안해도 된다. STEP 2 코드 한번에 작성.</STEP 2>       │
│                                                                                                                 │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'조건': {'지역': '서울', '기간': '2024.12.01 ~ 2025.04.10', '청약유형': '', 'keyword': '', 'sh_url':           │
│ 'https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv=', '공고상태': ''}}.  │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Calculate max_pages and scrape the sh_notice                                                           
  from datetime import datetime                                                                                    
                                                                                                                   
  # Define the period                                                                                              
  start_date = datetime.strptime('2024.12.01', '%Y.%m.%d')                                                         
  end_date = datetime.strptime('2025.04.10', '%Y.%m.%d')                                                           
  duration_months = (end_date.year - start_date.year) * 12 + end_date.month - start_date.month                     
  max_pages = (duration_months // 3) + (1 if duration_months % 3 > 0 else 0)                                       
                                                                                                                   
  # Scrape the notices                                                                                             
  url = 'https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv='                
  scraped_results = sh_notice_scraper(url=url, keyword='', max_pages=max_pages)                                    
                                                                                                                   
  # Filter results for 공고명 containing specific terms                                                            
  filtered_results_step1 = [result for result in scraped_results if any(term in result['공고명'] for term in       
  ['발표', '합격자', '경쟁률', '결과'])]                                                                           
  print("Filtered results after Step 1:", filtered_results_step1)                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Filtered results after Step 1: [{'번호': '54', '청약유형': '장기전세주택', '공고명': '제2차 장기전세주택2 입주자 
모집 공고(2024.08.30. 공고) 추가 서류심사 예비입주자 발표', '공고게시일': '2025-03-28', '공고상태': '접수중', 
'발표일': '2025-03-21', '담당부서': '미리내집공급부', '링크': 
'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=286419'}, {'번호': '50', '청약유형':
'청년안심주택', '공고명': '2024년 3차 청년안심주택(공공임대) 입주자 모집공고(2024. 12. 23.) 최종 청약경쟁률 게시', 
'공고게시일': '2025-01-07', '공고상태': '접수마감', '발표일': '2025-04-30', '담당부서': '맞춤주택공급부', '링크': 
'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=283348'}, {'번호': '49', '청약유형':
'장기전세주택', '공고명': '제3차 장기전세주택2 입주자 모집 공고(2024.12.20.) 청약 접수 최종 결과', '공고게시일': 
'2025-01-06', '공고상태': '접수마감', '발표일': '2025-04-25', '담당부서': '공공주택공급부', '링크': 
'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=283265'}, {'번호': '40', '청약유형':
'장기전세주택', '공고명': '제44차 장기전세주택 입주자 모집 공고(2024.6.27. 공고) 당첨자 및 예비입주자 발표', 
'공고게시일': '2024-11-15', '공고상태': '접수마감', '발표일': '2024-11-15', '담당부서': '공공주택공급부', '링크': 
'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=281012'}, {'번호': '39', '청약유형':
'전세임대', '공고명': '2024년 신혼신생아 전세임대 입주자 수시모집(2024.7.19.) 입주대상자 발표(계약안내문 첨부)', 
'공고게시일': '2024-10-22', '공고상태': '접수마감', '발표일': '2024-10-22', '담당부서': '전세주택공급부', '링크': 
'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=280246'}]

Out: None

[Step 0: Duration 11.99 seconds| Input tokens: 2,527 | Output tokens: 522]

...

[Step 1: Duration 8.74 seconds| Input tokens: 6,681 | Output tokens: 766]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(final_filtered_results)                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: [{'번호': '54', '청약유형': '장기전세주택', '공고명': '제2차 장기전세주택2 입주자 모집 
공고(2024.08.30. 공고) 추가 서류심사 예비입주자 발표', '공고게시일': '2025-03-28', '공고상태': '접수중', '발표일': 
'2025-03-21', '담당부서': '미리내집공급부', '링크': 
'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=286419'}, {'번호': '50', '청약유형':
'청년안심주택', '공고명': '2024년 3차 청년안심주택(공공임대) 입주자 모집공고(2024. 12. 23.) 최종 청약경쟁률 게시', 
'공고게시일': '2025-01-07', '공고상태': '접수마감', '발표일': '2025-04-30', '담당부서': '맞춤주택공급부', '링크': 
'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=283348'}, {'번호': '49', '청약유형':
'장기전세주택', '공고명': '제3차 장기전세주택2 입주자 모집 공고(2024.12.20.) 청약 접수 최종 결과', '공고게시일': 
'2025-01-06', '공고상태': '접수마감', '발표일': '2025-04-25', '담당부서': '공공주택공급부', '링크': 
'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=283265'}]

[Step 2: Duration 2.36 seconds| Input tokens: 11,719 | Output tokens: 822]

In [ ]:
from src.tools import YouthNoticeScraper

model = HfApiModel()
model = OpenAIServerModel(model_id='gpt-4o')
youth_notice_scraper = YouthNoticeScraper()
youth_agent =ToolCallingAgent(
    description=
    """
    youth_notice_scraper를 사용해서 웹사이트의 게시판 글 목록을 스크래핑하고 조건(예: 제목에 '공공임대' 포함, 기간이 '2024-01-01 ~ 2025-05-05', 공고명에 포함되어야 하는 keyword '공공')에 맞게 게시글 정보를 정리해서 반환하는 Tool Calling agent.
    """,
    tools=[youth_notice_scraper],
    model=model,
  #  use_e2b_executor=False,
    max_steps = 3
)
opt = {'지역' : "서울",
      '기간' : "2024.07.01 ~ 2025.04.10",
      '청약유형' : "청년안심주택",
      'keyword' : "공공",
      'sh_url' : "https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv="
      }

task_prompt = f"""
청년안심주택 모집 공고를 youth_notice_scraper을 찾고 조건(지역, 기간)에 맞는 게시글들을 찾아서 결과를 정리해.
참고 사항 1: max_pages 수는 조건 기간이 5개월 당 페이지 4개를 고려해서 tool 호출.
"""
youth_final_answer = youth_agent.run(task=task_prompt, additional_args={'조건':{'지역':opt['지역'],'기간':opt['기간'],'keyword':opt['keyword']}}, stream=False)
print(youth_final_answer)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 청년안심주택 모집 공고를 youth_notice_scraper을 찾고 조건(지역, 기간)에 맞는 게시글들을 찾아서 결과를 정리해.   │
│ 참고 사항 1: max_pages 수는 페이지4개 당 5개월을 고려해서 tool 호출.                                            │
│                                                                                                                 │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'지역': '서울', '기간': '2024.12.01-2025.04.10', 'keyword': '공공'}.                                           │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'youth_notice_scraper' with arguments: {'max_pages': 5}                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |{'공고명': '|민간임대] 영등포구청역 포레나 당산 추가모집공고', '공고게시일': '2025-03-24', 
'모집마감일': '2025-03-27', '청약유형': '민간임대 청년안심주택', '시행사': '(주)당산동청년주택피에프브이', '게시글 
ID': 6200, '상세 URL': 'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6200&menuNo=400008'}, 
{'공고명': '|민간임대] 서울대입구역 BX201 추가모집공고', '공고게시일': '2025-03-24', '모집마감일': '2025-03-28', 
'청약유형': '민간임대 청년안심주택', '시행사': '라온스테이 2030 역세권청년주택 (박정호 외 3인)', '게시글 ID': 6201,
'상세 URL': 'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6201&menuNo=400008'}, {'공고명': 
'|민간임대] 노량진역 더클래식 동작 추가모집공고', '공고게시일': '2025-03-19', '모집마감일': '2025-03-24', 
'청약유형': '민간임대 청년안심주택', '시행사': '더클래식 동작', '게시글 ID': 6198, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6198&menuNo=400008'}, {'공고명': '|민간임대] 노량진역
더써밋타워 추가모집공고', '공고게시일': '2025-03-19', '모집마감일': '2025-03-24', '청약유형': '민간임대 
청년안심주택', '시행사': '코레이트노량진역청년주택 유한회사', '게시글 ID': 6199, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6199&menuNo=400008'}, {'공고명': '|민간임대] 길동역 
길동생활(B동) 추가모집공고', '공고게시일': '2025-03-13', '모집마감일': '2025-03-17', '청약유형': '민간임대 
청년안심주택', '시행사': '(주)대한제37호위탁관리부동산투자회사', '게시글 ID': 6196, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6196&menuNo=400008'}, {'공고명': '|민간임대] 길동역 
길동생활(A동) 추가모집공고', '공고게시일': '2025-03-13', '모집마감일': '2025-03-17', '청약유형': '민간임대 
청년안심주택', '시행사': '(주)대한제37호위탁관리부동산투자회사', '게시글 ID': 6195, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6195&menuNo=400008'}, {'공고명': '|민간임대] 노량진역
더클래식 동작 추가모집공고', '공고게시일': '2025-03-07', '모집마감일': '2025-03-10', '청약유형': '민간임대 
청년안심주택', '시행사': '더클래식 동작', '게시글 ID': 6192, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6192&menuNo=400008'}, {'공고명': '|민간임대] 등촌역 
아르체움 등촌 최초모집공고', '공고게시일': '2025-03-07', '모집마감일': '2025-03-17', '청약유형': '민간임대 
청년안심주택', '시행사': '윤호표 외 8인 (대현빌딩)', '게시글 ID': 6193, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6193&menuNo=400008'}, {'공고명': '|민간임대] 
건대입구역 더포디엄830 추가모집공고', '공고게시일': '2025-03-06', '모집마감일': '2025-03-10', '청약유형': '민간임대
청년안심주택', '시행사': '(주)엘앤케이이스테이트', '게시글 ID': 6191, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6191&menuNo=400008'}, {'공고명': '|민간임대] 
영등포구청역 포레나 당산 추가모집공고', '공고게시일': '2025-03-05', '모집마감일': '2025-03-08', '청약유형': 
'민간임대 청년안심주택', '시행사': '(주)당산동청년주택피에프브이', '게시글 ID': 6190, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6190&menuNo=400008'}, {'공고명': '|민간임대] 연신내역
루체스테이션 추가모집공고', '공고게시일': '2025-03-04', '모집마감일': '2025-03-08', '청약유형': '민간임대 
청년안심주택', '시행사': '(주)위드웰에셋', '게시글 ID': 6188, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6188&menuNo=400008'}, {'공고명': '|민간임대] 노량진역
더써밋타워 추가모집공고', '공고게시일': '2025-03-04', '모집마감일': '2025-03-07', '청약유형': '민간임대 
청년안심주택', '시행사': '코레이트노량진역청년주택 유한회사', '게시글 ID': 6189, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6189&menuNo=400008'}, {'공고명': '|민간임대] 발산역 
엘크루 발산 추가모집공고', '공고게시일': '2025-02-24', '모집마감일': '2025-02-28', '청약유형': '민간임대 
청년안심주택', '시행사': '펠리즈 타워 (윤석표 외 2인)', '게시글 ID': 6187, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6187&menuNo=400008'}, {'공고명': '|민간임대] 
숭곡초교역(예정) 성북 펠릭스 추가모집공고', '공고게시일': '2025-02-21', '모집마감일': '2025-02-26', '청약유형': 
'민간임대 청년안심주택', '시행사': '꿈꾸는 이상(주)', '게시글 ID': 6184, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6184&menuNo=400008'}, {'공고명': '|민간임대] 우장산역
우장산역 해링턴타워 추가모집공고', '공고게시일': '2025-02-20', '모집마감일': '2025-02-24', '청약유형': '민간임대 
청년안심주택', '시행사': '주식회사 선우', '게시글 ID': 6183, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6183&menuNo=400008'}, {'공고명': '|민간임대] 강변역 
비바힐스강변 추가모집공고', '공고게시일': '2025-02-20', '모집마감일': '2025-02-24', '청약유형': '민간임대 
청년안심주택', '시행사': '정지량 외 6명', '게시글 ID': 6181, '상세 URL': 
'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6181&menuNo=400008'}, {'공고명': '|민간임대] 노량진역
더클래식 동작 추가모집공고', '공고게시일': '2025-02-19', '모집마감일': '202

[Step 0: Duration 4.69 seconds| Input tokens: 1,192 | Output tokens: 19]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "Here are the 청년안심주택 모집 공고 that meet your     │
│ criteria:\n\n1. **[공공임대] 2024년 3차 청년안심주택 모집공고**\n   - 게시일: 2024-12-23\n   - 마감일:          │
│ 2025-01-02\n   - 시행사: SH 맞춤주택공급부\n   - [상세                                                          │
│ 링크](https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6152&menuNo=400008)\n\nThese postings are    │
│ located in Seoul and are within the specified date range of 2024.12.01 to 2025.04.10, and include the keyword   │
│ '공공'. Only relevant entries were selected based on the criteria you provided."}                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Here are the 청년안심주택 모집 공고 that meet your criteria:

1. **[공공임대] 2024년 3차 청년안심주택 모집공고**
   - 게시일: 2024-12-23
   - 마감일: 2025-01-02
   - 시행사: SH 맞춤주택공급부
   - [상세 링크](https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6152&menuNo=400008)

These postings are located in Seoul and are within the specified date range of 2024.12.01 to 2025.04.10, and 
include the keyword '공공'. Only relevant entries were selected based on the criteria you provided.

[Step 1: Duration 3.35 seconds| Input tokens: 9,434 | Output tokens: 212]

Here are the 청년안심주택 모집 공고 that meet your criteria:

1. **[공공임대] 2024년 3차 청년안심주택 모집공고**
   - 게시일: 2024-12-23
   - 마감일: 2025-01-02
   - 시행사: SH 맞춤주택공급부
   - [상세 링크](https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6152&menuNo=400008)

These postings are located in Seoul and are within the specified date range of 2024.12.01 to 2025.04.10, and include the keyword '공공'. Only relevant entries were selected based on the criteria you provided.
